<img src="https://www.epfl.ch/about/overview/wp-content/uploads/2020/07/logo-epfl-1024x576.png" style="padding-right:10px;width:140px;float:left"></td>
<h2 style="white-space: nowrap">Neural Signals and Signal Processing (NX-421)</h2>
<hr style="clear:both"></hr>


Welcome to the laboratory computers for the course "Neural signals and signal processing". Today, you will see how to preprocess fMRI data as well as play around with diffusion weighted imaging to compute a simple tractography.

In [ ]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

!pip install antspyx
import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json

In [ ]:
################
# Start FSLeyes (very neat tool to visualize MRI data of all sorts) within Python
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Careful ! 💡</b></p>
<p style='text-indent: 10px;'>
    Some of the fMRI preprocessing you will see below leverage the anatomical MRI being preprocessed, which you know how to do from week 2. In case of doubt, go back to the previous week to see what you might need.</p>
</span>
</div>

This notebook is structured into two distinct parts:
- **Part1: fMRI preprocessing**
- **Part2: diffusion data and tractography generation**

# Part 1: fMRI preprocessing

You are now familiar with the few steps of preprocessing revolving around the T1 anatomical file. The main preprocessing starts now, with the functional data. 

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>💡 Do not forget QC! 💡</b></p>
<p style='text-indent: 10px;'>
    As always, in all your steps visualize the effect of what you're doing. This is the easiest way to check that what you're doing is actually having an effect and better yet: a <b>correct</b> effect!</p>
</span>
</div>

Note that one assumption with fMRI preprocessing is that you've already conducted the anatomical preprocessing. In particular, the two main steps you will need today (repeat it from lab 2) before launching the lab:
- T1 skull-stripping (use BET)
- T1 segmentation (use FAST)
Do it on sub-001, dataset with ID ds004226.


## 1.0 Problematic volumes removal

A problem can arise in fMRI. To showcase this, please execute the cell below (we'll show you something from another dataset just to drive our point home).

In [ ]:
import subprocess
dataset_demo = 'ds000114'
subject_demo = '01'

sample_path = "/home/jovyan/Data/dataset"
mkdir_no_exist(sample_path)

# Download one subject's data from each dataset
bids_root_demo = op.join(os.path.abspath(""),sample_path, dataset_demo)
preproc_root_demo = op.join(bids_root_demo, 'derivatives')

subprocess.run(["openneuro-py", "download", "--dataset", dataset_demo, "--target-dir", bids_root_demo, "--include", 'sub-{}'.format(subject_demo)], check=True)

We've downloaded one functional volume from another dataset, because the phenomenon is really visible in this dataset. 
Before going any further in this tutorial, let's open up our data and have a look at them. <u>You should always look at your data before conducting any sort of analysis</u>. See if you find anything at all that looks strange. You should look for

- [ ] Volumes moving in space (ie: head motion)
- [ ] Non homogeneities that do not seem to be coming from brain activity
- [ ] Changes in *overall* contrast

To open the volume of interest in FSL eyes, simply run:

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(bids_root_demo, 'sub-01', 'ses-test', 'func', 'sub-01_ses-test_task-fingerfootlips_bold.nii.gz'))

Did you find anything?
If so, what volumes would you remove, approximately?

### 1.0.1 Field stabilization

The scanner's field takes some time to settle. You probably noticed that the initial volume had a high contrast that quickly decayed to some baseline? It is precisely caused by the scanner's field settling.

These scans are called *non-stationary volumes*, because they are acquired while the B0 field is not yet not stable.

There's little to be done in this regard; we can only throw away the volumes that are contaminated in this specific case, to ensure this change in global signal does not drive our analysis.

If we're being really formal, a higher overall contrast can be detected by looking at the mean voxel value in each volume, like so:

In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib

plt.plot(nib.load(op.join(bids_root_demo, 'sub-01', 'ses-test', 'func', 'sub-01_ses-test_task-fingerfootlips_bold.nii.gz')).get_fdata().mean(axis=(0,1,2)))
plt.xlabel('Time (volume)')
plt.ylabel('Mean voxel intensity')

Clearly, while we could throw away only the first volume and the second one for good measure, we'll **choose** to discard the first 10 volumes, to be absolutely sure the scanner is settled.

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> Modern techs to the rescue</b></p>
<p style='text-indent: 10px;'>
    Most scanners nowadays actually acquire few scans to help the B0 field settle. These are called dummy scans; the scanner acquires them but throws them away, meaning that you end up with a result that is settled. You should pay attention however if you ever analyze older datasets, as they will not benefit from these new techs obviously!</p>
</span>
</div>

In [ ]:
file_to_trim = glob.glob(op.join(bids_root_demo, 'sub-01', 'ses-test', 'func', 'sub-01_ses-test_task-fingerfootlips_bold.nii.gz'))[0]
mkdir_no_exist(preproc_root_demo)
mkdir_no_exist(op.join(preproc_root_demo, 'sub-01'))
mkdir_no_exist(op.join(preproc_root_demo, 'sub-01', 'ses-test'))
mkdir_no_exist(op.join(preproc_root_demo, 'sub-01', 'ses-test','func'))
output_target = op.join(preproc_root_demo, 'sub-01', 'ses-test', 'func', 'sub-01_ses-test_task-fingerfootlips_bold_settled.nii.gz')

############################
# Solution
# Because we start at 0, we begin at the 11th volume, excluding 10 volumes: we should start at number 10.
# There are 184 volumes. We remove 11 volumes, so we're left with 174 volumes.
###########################

# For this, knowing that there are originally 184 volumes and that you want to throw away the first 10, please fill in
# the following variables
start_vol = 10 # Where should we start? (First volume is 0, not 1 !)
number_of_volumes = 174 # How many volumes should we keep?

# Function creating new nifti with trimmed number of volumes
fslroi(file_to_trim, output_target, str(start_vol), str(number_of_volumes))
print(f"You kept {nib.load(output_target).shape[-1]} volumes.")

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b>🏠 Take-home message 🏠</b></p>
<p>Always look at your data!</p>
</span>
</div>


Let's go back to our original dataset now. :)

## 1.2 Motion correction

Motion correction here specifically means trying to make it such that a given voxel describes the same brain position in all volumes.

To illustrate why it might be a good idea, let's have a look at the functional data of our participant. Watch the movie. Do you notice anything strange?

<center><img src="imgs/motion.gif"/>
    <p style="text-align:center;"><i>You might want to pay attention to the axial view (right)</i></p></center>


The volumes tend to move a bit around, don't they?
<an example moment of motion>
    
This is a problem. Indeed, when we talk of a given voxel, our hope for analysis is that it represents a specific coordinate of anatomy. Imagine if you were trying to find your way with Google Maps, but every now and then the houses would suddenly all move by one kilometer! Would not be so easy to get to the right address, would it? Well, here it's the same. We want that a given (X, Y, Z) position describes always the same portion of the brain, otherwise our analysis will simply not work.

But because of motion, this is not the case.
    
This is one of the core issues of fMRI: the participant simply moved, ever so slightly, during the acquisition. As a consequence, well, we have a recording of a moving participant. This is not a rare phenomenon: imagine having to keep your head perfectly still for several minutes and you'll quickly understand that it is **hard**!
    
Still, we would like to do something about it. This is where motion correction steps in. There are two sides to motion correction. The first, which we'll cover here, attempts to put all volumes back in alignment, so that a given position is indeed consistently describing the same anatomical part for all volumes. The second, which you'll see next week, attempts to correct for the consequences of motion on the magnetic field.
<br><br>
Do you remember Ducky? Well, imagine now that our dear duck has a rare shaking disease.
    <br><img src="imgs/ducky/shakyducky.png" style="width:auto;height:500px;"/>
<br>If we take several consecutive pictures of Ducky, it won't be as aligned as it should be
    <br><img src="imgs/ducky/duckies_before_reg.png" style="width:900px;height:auto;"/>
<br>To correct this, I can apply the idea we used in normalization. Let's pick one Ducky image as reference. Then, all other images of Ducky will be registered to this Ducky
    <br><img src="imgs/ducky/duckies_reg.png" style="width:900px;height:auto;"/>
    <br><img src="imgs/ducky/duckies_after_reg.png" style="width:900px;height:auto;"/>
<br>Now, what if I want to remember how much Ducky had moved ? Well, I can remember the parameters of the transformation I had to apply to align the volumes. This fully encapsulates the motion information.

This is precisely what motion correction sets out to achieve. For this, we need first to define a reference, if possible in fMRI space and that would not require too much transformations. Which option(s) seem reasonable to you?:

In [ ]:
interactive_MCQ(3,1)

It turns out the first two options are usually equivalent. Because it saves us one pass of average computation, we will choose the first option: picking a volume and using it as reference! Which one do you think would be good?

In [ ]:
interactive_MCQ(3,2)

Now, let us perform this step, on our **first** dataset (the one without fieldmaps). In FSL, we use <a href="https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/MCFLIRT">MCFLIRT</a> to perform this correction.
<br>
    
<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Pay attention ! 💡</b></p>
<p style='text-indent: 10px;'>
    By default, MCFLIRT selects the middle volume of the EPI serie as reference to which other volumes are realigned.</p>
</span>
</div>

In [ ]:
from fsl.wrappers import mcflirt

dataset_id = 'ds004226'
subject = '001' 

sample_path = "/home/jovyan/Data/dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

subprocess.run(["openneuro-py", "download", "--dataset", dataset_id, "--target-dir", bids_root, "--include", 'sub-{}'.format(subject)], check=True)

#mkdir_no_exist(bids_root)
mkdir_no_exist(deriv_root)
mkdir_no_exist(preproc_root)
mkdir_no_exist(op.join(preproc_root, 'sub-001'))
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-001', 'anat'))

path_original_data = op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
path_moco_data = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco')
mcflirt(infile=path_original_data,o=path_moco_data, plots=True, report=True, dof=6, mats=True)

Okay! So, what do we have now?

In [ ]:
print_dir_tree(bids_root, max_depth=5)

In the functional folder, notice that we have two new files:

**sub-001_task-sitrep_run-01_bold_moco.nii.gz**:
- This is your motion-corrected EPI time series (4D NIfTI).
- Each volume in the initial run has been spatially realigned to the same reference volume (the middle volume).
- This is the dataset you’ll use for further preprocessing and analysis.

**sub-001_task-sitrep_run-01_bold_moco.par**:
- This file contains the motion parameters, one line per volume.
- Typically 6 numbers per line: 3 rotations (pitch, yaw, roll — around x, y, z axes) + 3 translations (in mm along x, y, z axes)
- These numbers describe the head movement at each timepoint with respect to the reference volume.

Notice as well a new directory!

**sub-001_task-sitrep_run-01_bold_moco.mat/**
- This directory contains one .mat file per volume (each containing 4×4 matrices).
- Each file is a transformation matrix describing the exact affine transformation used to map the indexed volume to the reference volume.
- While the .par file show head movement, the matrices enact the transformation for realignment.

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Pay attention ! 💡</b></p>
<p style='text-indent: 10px;'>
    The motion parameters and the transformation matrices are related, but they are not exactly the same thing. While you can recover one from the other, it is not trivial. Applying the transformation matrix to a volume will put it 'in alignment' as you've done with FLIRT. However the motion parameters cannot be applied directly. Loosely, the motion parameters describe how you would move if you first applied a rotation along x, then along y, then along y, followed by transition along x, then transition along y, then transition along z. This ordering of transformations is **not** really what happens with the transformation matrices. It is a convention adopted by FSL to make it easier to decouple transformations and rotations in the motion parameter analysis; it is therefore a <b>convenience</b>.
    <u>Do not confuse transformation matrices and motion parameters</u>!</p>
</span>
</div>

Before going <u>any further</u>, go and have a look at the corrected timeseries.

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(path_original_data)
fsleyesDisplay.load(path_moco_data)

Did mcflirt help correct motion? Are you convinced it did somewhat a proper job?
<br>
It's actually not too easy to tell right? Well, let's see if we can figure something out to ease our quality control!

#### 1.2.1 Motion parameters and degrees of freedom

We told you earlier that motion parameters can be used to estimate the motion along every axis.

In our invocation of mcflirt, notice the following:
```python
mcflirt(..., dof=6)
```
dof stands for <i><b>d</b>egrees <b>o</b>f <b>f</b>reedom</i>, it really means what kind of transformation we wish to apply. In a 3D transformation, we have 3 axis:<br>
<img src="imgs/3d_axis.png"/><br>
Along each axis, we can apply one transformation. Because we apply here only **affine** transformations, we can choose any transformation from:
- Translation along the axis
- Rotation along the axis
- Shear along the axis
- Scale along the axis

Together, you can see this gives in total **12** DOF.
We've chosen 6 DOFs, which is the standard choice: we want only to translate and rotate around the volumes, since they've been displaced by motion.

#### 1.2.2 Looking at the resulting correction parameters
Recall the motion parameters are stored in the .par file produced by MCFLIRT. Notice that since each volume moved differently, we have one transformation per volume, thus one set of motion parameters per volume as well. We provide you with a way to load these parameters:

In [ ]:
def load_mot_params_fsl_6_dof(path):
    return pd.read_csv(path, sep='  ', header=None, 
            engine='python', names=['Rotation x', 'Rotation y', 'Rotation z','Translation x', 'Translation y', 'Translation z'])

mot_params = load_mot_params_fsl_6_dof(op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.par'))
mot_params

Based on **translation on X alone**, can you find perhaps a volume which exceeds with respect to the **preceding volume** a 0.2 mm displacement?

In [ ]:
# write your code here to inspect quickly the translation on X :) # Hint: look at the documentation for np.diff this may come in handy!
#%matplotlib inline

# Getting the translation is easy
trans_x = mot_params["Translation x"]
# Now, we want a 0.2mm with respect to previous frame:
disp_x = np.diff(trans_x)
# Lastly, we can ask for displacements (in absolute value) above 0.2mm and plot it to be clear:
threshold=0.2
plt.plot(np.abs(disp_x))
plt.hlines(threshold, 0, 370,colors='black', linestyles='dashed', label='FD threshold')
plt.xlabel("Volumes")
plt.ylabel("Framewise translation displacement (mm)")
plt.show()

# There are basically (if looking only along X translation) no frame displacement above 0.2mm!

Some metrics have been created, to compute the displacement of a frame compared to the preceding frame: this is the **frame-wise displacement**. While considering the x-translation can be indicative of motion, it omits displacement in other directions. Here the **frame-wise displacement** allows to quantify overall displacements. <br>(see <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3254728/">Power, Jonathan D., et al. "Spurious but systematic correlations in functional connectivity MRI networks arise from subject motion." Neuroimage 59.3 (2012): 2142-2154.</a> for more details).<br>
We can use this one to extract an aggregate measure of motion for all volumes. 

In [ ]:
def compute_FD_power(mot_params):
    framewise_diff = mot_params.diff().iloc[1:]

    rot_params = framewise_diff[['Rotation x', 'Rotation y', 'Rotation z']]
    # Estimating displacement on a 50mm radius sphere
    # To know this one, we can remember the definition of the radian!
    # Indeed, let the radian be theta, the arc length be s and the radius be r.
    # Then theta = s / r
    # We want to determine here s, for a sphere of 50mm radius and knowing theta. Easy enough!
    
    # Another way to think about it is through the line integral along the circle.
    # Integrating from 0 to theta with radius 50 will give you, unsurprisingly, r0 theta.
    converted_rots = rot_params*50
    trans_params = framewise_diff[['Translation x', 'Translation y', 'Translation z']]
    fd = converted_rots.abs().sum(axis=1) + trans_params.abs().sum(axis=1)
    return fd

fd = compute_FD_power(mot_params).to_numpy()

In [ ]:
threshold = np.quantile(fd,0.75) + 1.5*(np.quantile(fd,0.75) - np.quantile(fd,0.25))

In [ ]:
#%matplotlib inline
plt.plot(list(range(1, fd.size+1)), fd)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
plt.hlines(threshold, 0, 370,colors='black', linestyles='dashed', label='FD threshold')
plt.legend()
plt.show()

Okay great, but what if we want to know which volumes are actually above threshold? Simply run the cell below!

In [ ]:
print(f"Indices of frames above threshold: {(np.where(fd > threshold)[0] + 1).tolist()}")

So, you now know which volumes might present motion that is worth checking. Go back to FSLeyes and contrast the uncorrected volumes with the corrected ones. Can you see what sort of motion was problematic and was eliminated?

## 1.3 Motion-correction: conclusions

Motion correction should always be conducted. As you've seen, it is extremely easy to do and has many benefits. However it is not infaillible. High motion tends to cause non linear effects in the signal that simple motion correction above cannot correct since it has no awareness of the magnetic field. <br>
<br> Motion parameters can, in this case, come to our rescue. As they represent the effect of motion, including them in our modeling to try and correct the signal can help. One could for example include this information in a General Linear Model to regress out the signal of these volumes (censoring) from overall timeseries. ➡️ More on this next week!

## 1.4 Where are we?

So, let's see what we have done so far:

<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Anatomical</th><td></td><td></td></tr>
    <tr><td></td><td style='text-align:justify;'>Skull stripping</td><td style='text-align:justify;'>Removing skull and surrounding tissues to keep only the brain</td><td style='text-align:justify;'>BET</td></tr>
    <tr><td></td><td style='text-align:justify;'>Segmentation</td><td style='text-align:justify;'>Segmenting brain tissues based on their contrasts</td><td style='text-align:justify;'>FAST</td></tr>
    <tr><td></td><td style='text-align:justify;'>Normalization</td><td style='text-align:justify;'>Mapping participant's brain to a reference brain, making its orientation and scale match so that comparison across participants become feasible.</td><td style='text-align:justify;'>FLIRT</td></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>First few volumes removal</td><td style='text-align:justify;'>Removing volumes for which the B0 field is still not stable and that could contaminate all our data if left unchecked.</td><td style='text-align:justify;'>fslroi</td></tr>
    <tr><td></td><td style='text-align:left;'>Motion correction</td><td style='text-align:justify;'>Realignment of fMRI volumes to a common reference - typically one volume or the average of the volumes - to correct for inter-volume motion. The extracted motion parameters can be used for subsequent analysis (see GLM in two weeks!)</td><td style='text-align:justify;'>MCFLIRT (which is one suboption of FLIRT in fact)</td></tr>
</table>

You will see some additional steps next week (slice-timing and field unwarping to be specific).

## 1.5 Coregistration of functional to anatomical

You have seen coregistration last week, when you were trying to align the T1 to MNI152, both manually and algorithmically. In the specific case of putting a T1 anatomical in a template space (such as MNI), we call it <b>normalization</b>, because we...Normalize it !

You've also seen above with motion-correction coregistration of EPI volumes (functional data) to each other to correct motion.

But what if you wanted to put the functional data overlayed on the anatomy, to know more precisely which parts of the brain are activated?

You will see the last important ingredient to do this: coregistration between functional and anatomical.

In this specific case, it is a registration between images of different modalities. We want to register fMRI (EPI) to an anatomical image (T1). There are several reasons for this. The first that comes to mind is that if you overlay your fMRI on the anatomy, you can of course reason much more easily on where you are in the brain, what activations you might be looking at and so forth. Imagine a participant has a brain lesion visible on the anatomy and you want to see how this reflects on the fMRI. Being able to put the two together would make it much easier, would it not?

This is the first reason behind coregistration.

The second is because of normalization, again. Assume you want to compare all fMRI data of participants. Clearly, putting all of them into a common reference frame is a bit trickier, because of how noisy and low-resolution the data is, right? But you know how to map the anatomical to this common space with excellent accuracy, and you've saved this transformation earlier.
If you could figure out how to go from the fMRI space to anatomical, clearly the problem would be solved! You'd only have then to apply the transformation from anatomical to common space and be done with it.


Computing the fMRI space to anatomical transformation is precisely the goal of coregistration.
<br><br>
To do this step, we will use a wonderful command: <a href="https://web.mit.edu/fsl_v5.0.10/fsl/doc/wiki/FLIRT(2f)UserGuide.html#epi_reg">epi_reg</a> ! As the name states, it is a command to register an EPI. Hard to make it clearer huh? 

### 1.5.1 Using epi_reg to do the EPI registration

Notice that we want to compute the transformation to use for coregistration.

In [ ]:
interactive_MCQ(3,3)

Your task is simple. You should:
- Fill in the name of the EPI source. It should be the **motion-corrected** EPI that you corrected using MCFLIRT. If you want to use a single volume, set the use_first_vol variable to True!
- Fill in the path to the whole head T1 image (**before** skull stripping was conducted!)
- Fill in the path to the skull-stripped T1 image (**after** skull stripping was conducted!)

Prior to using epi_reg make sure that you have generated both the skull stripped T1 and its tissue segmentation as they will be used in the following cells!

In [ ]:
# Important: Remember to generate skull stripped T1 and perform tissue segmentation if you did not yet !!! Hint: Use skull stripping and segmentation function from Lab 2!

# Generate skull stripped T1
anatomical_path = op.join(bids_root, 'sub-001', 'anat', 'sub-001_T1w.nii.gz')
betted_brain_path = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w')
os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, ''))

# Generate segmented tissues
[os.remove(f) for f in glob.glob(op.join(preproc_root, 'sub-001', 'anat', '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w_fast')
fast(imgs=[betted_brain_path], out=segmentation_path, n_classes=3)

<div class="warning" style='background-color:#C1ECFA; color: #112A46; border-left: solid darkblue 4px; border-radius: 4px; padding:0.7em;'>
<span>
<p style='margin-top:1em; text-align:center'><b> 💡 Pay attention ! 💡</b></p>
<p style='text-indent: 10px;'>
    Make sure that the whole head T1 and the skull-stripped T1 have the same orientation.
For example, if you ran fsl_anat to extract the brain (which is fine), FSL will change in the headers the orientation of the T1 before skull-stripping. As a consequence, the brain-extracted T1 no longer has the same orientation as the original T1. If you display them on top of each other, they are perfectly matched, but not from the perspective of the <b>headers</b>, which can play nasty tricks on you when performing coregistration.</p>
</span>
</div>

First, we'll show you the documentation of epi_reg:

In [ ]:
subprocess.run(['epi_reg'])

epi_reg has one peculiarity. If you launch it on a 4D volume, it will truncate your result to the first volume, because it expects a *single* EPI volume. We should thus first extract a single volume from our EPI, and then call epi_reg on it. We do that for you below.

If you want to run with 4D volume to see the result and the warning, set use_single_vol to False

In [ ]:
from fsl.wrappers import epi_reg

#################
# Solution
# We use the motion corrected EPI 
##################

# We use the motion corrected EPI
epi_source = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco')
whole_t1 = op.join(bids_root, 'sub-001', 'anat', 'sub-001_T1w')
skull_stripped_t1 = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w')
output_path = op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_bbr')
ref_vol_name =  op.join(preproc_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco_vol_middle')

use_single_vol = True

if use_single_vol:
    # Extract the middle volume with fslroi as we've seen before :)
    fslroi(epi_source, ref_vol_name, str(182), str(1))
    # Call epi_reg
    subprocess.run(['epi_reg','--epi={}'.format(ref_vol_name), '--t1={}'.format(whole_t1), '--t1brain={}'.format(skull_stripped_t1), '--out={}'.format(output_path)])
else:
    subprocess.run(['epi_reg','--epi={}'.format(epi_source), '--t1={}'.format(whole_t1), '--t1brain={}'.format(skull_stripped_t1), '--out={}'.format(output_path)])


Notice how FAST is ran?
This is because the specific coregistration cost (boundary-based registration, BBR) uses the anatomical white-matter tissues from FAST. If no such tissue is provided to the function, it re-runs FAST to obtain it and use it. If you've already done anatomical segmentation, clearly there's no need to redo it right?
In particular, imagine if you had to yourself correct the white matter with the help of an expert because somehow FSL did a poor job on your data. Clearly you'd like to have this one used instead of the result from FAST, right?

Well- you can! We just need a new option in the epi_reg command:
```python
epi_reg(...,wmseg=path_to_your_white_matter_segmentation)
```
Remember from last week, which T1 file corresponds to the white matter, between pve_0, pve_1 and pve_2 ? Put it as your white matter segmentation. (Note: of course, you will need to re-do your T1 segmentation before you can feed it here - a good moment to remember the previous week and revisit FAST and BET ;) )

In [ ]:
###############
# Solution
# White matter corresponds to pve_2.
##############

white_matter_segmentation = op.join(preproc_root, 'sub-001', 'anat', 'sub-001_T1w_fast_pve_2.nii.gz') # We provide the white matter segmentation

subprocess.run(['epi_reg','--epi={}'.format(ref_vol_name), '--t1={}'.format(whole_t1), '--t1brain={}'.format(skull_stripped_t1), 
                '--out={}'.format(output_path),
               '--wmseg={}'.format(white_matter_segmentation)])

Let's overlay the two (EPI and anatomical) on top of each other to visualize the quality of the coregistration! (Note: You can change the opacity of either volumes to see better whether they overlap!)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(skull_stripped_t1)
fsleyesDisplay.load(output_path)

Now, how do we *know* if the registration is good or bad?
Well, there are several things to watch out for, but here are some main leads:
- Is the functional in the right orientation?
- Are the ventricles correctly aligned?
- Are the boundaries of the EPI more or less matching the anatomical?

➡️ You can also check how the white matter of the EPI matches your anatomical's white matter provided you have sufficient resolution

#### 1.5.2 Some cleanup
If you have a look, you might notice that perhaps your directory got filled with many files. These are temporary files, created but uncorrectly not eliminated by epi_reg. The following should help:

In [ ]:
def cleanup_epi_reg(path_to_clean):
    patterns = ['*_fast_*', '*_fieldmap*']
    for p in patterns:
        files = glob.glob(op.join(path_to_clean, p))
        for f in files:
            os.remove(f)

In [ ]:
cleanup_epi_reg(op.join(preproc_root, 'sub-001', 'func'))

In [ ]:
print_dir_tree(bids_root, max_depth=5)

## 1.6 Smoothing
All these transforms are not exactly perfect. As you've seen in class, a step of smoothing is typically applied, with the size of the smoothing being dependent on your application, starting resolution etc.
The idea of smoothing is really that, as you're averaging, hopefully you increase the signal to noise ratio. <br>
A side-effect is that finest patterns of activation will be lost in the averaging (we can't have everything: there's no free lunch).

With FSL, smoothing is rather easy to do. However, one thing which is important is the size of your filter.
Different softwares might use different conventions. For MRI, it is typical to talk about FWHM (Full-width at half maximum), expressed in mms.

FSL, however, takes as input in sigma instead of FWHM. The conversion is easy fortunately:

$$ \sigma = \frac{FWHM}{2.3548}$$

Here for example would be the smoothing command for 6mm FWHM smoothing:

In [ ]:
cmd = 'fslmaths {} -s {} {}_smoothed-6mm'.format(output_path, 6/2.3548, output_path)
subprocess.run(['fslmaths',output_path, '-s', str(6/2.3548), '{}_smoothed-6mm'.format(output_path)])

Let's observe what we have now:

In [ ]:
fsleyesDisplay.load(output_path + '_smoothed-6mm')

Do you feel as though the signal-to-noise ratio was improved?

## 1.7 MRI + fMRI preprocessing: summary

So, these were all the steps you were meant to study this week. Next week, we'll present advanced steps of fMRI, but they are not always conducted unlike those we've shown you above which should always be considered.

You should know by now: preprocessing is extremely important and you will likely spend a lot of time on it. Decisions in preprocessing will affect your analysis, so do not take this step lightly, it is <u>critical</u> to do it as well as possible!

<u>Always perform quality control to ensure everything is okay!</u>

Let's review one last time the different steps you've studied and which FSL tool(s) you used to do it:
<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Anatomical</th><td></td><td></td></tr>
    <tr><td></td><td style='text-align:justify;'>Skull stripping</td><td style='text-align:justify;'>Removing skull and surrounding tissues to keep only the brain</td><td style='text-align:justify;'>BET</td></tr>
    <tr><td></td><td style='text-align:justify;'>Segmentation</td><td style='text-align:justify;'>Segmenting brain tissues based on their contrasts</td><td style='text-align:justify;'>FAST</td></tr>
    <tr><td></td><td style='text-align:justify;'>Normalization</td><td style='text-align:justify;'>Mapping participant's brain to a reference brain, making its orientation and scale match so that comparison across participants become feasible.</td><td style='text-align:justify;'>FLIRT + FNIRT (from last week), or ANTs</td></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>First few volumes removal</td><td style='text-align:justify;'>Removing volumes for which the B0 field is still not stable and that could contaminate all our data if left unchecked.</td><td style='text-align:justify;'>fslroi</td></tr>
    <tr><td></td><td style='text-align:left;'>Motion correction</td><td style='text-align:justify;'>Realignment of fMRI volumes to a common reference - typically one volume or the average of the volumes - to correct for inter-volume motion. The extracted motion parameters can be used for subsequent analysis (see GLM next week!)</td><td style='text-align:justify;'>MCFLIRT (which is one suboption of FLIRT in fact)</td></tr>
    <tr><td></td><td style='text-align:left;'>Coregistration to anatomical</td><td style='text-align:justify;'>Putting the functional volumes in anatomical space</td><td style='text-align:justify;'>FLIRT (epi_reg being a specialized instance)</td></tr>
    <tr><td></td><td style='text-align:left;'>Smoothing</td><td style='text-align:justify;'>Allowing a bit of lee-way in the voxel's values to account for the imperfection of the registration</td><td style='text-align:justify;'>fslmath with smoothing operation</td></tr>
</table>

# Part 2: diffusion data and tractography generation

The preprocessing in DTI involves similar steps to what you saw in fMRI. We will thus tackle specific different steps to not repeat ourselves too much:
<center><img src="imgs/DTI_preprocessing.png" width="600"/></center>
<p  style="text-align: center;"><i>Image from <a href="https://www.researchgate.net/publication/311246309_Imaging_analysis_of_Parkinson's_disease_patients_using_SPECT_and_tractography">Son, Seong-Jin, Mansu Kim, and Hyunjin Park. "Imaging analysis of Parkinson’s disease patients using SPECT and tractography." Scientific reports 6.1 (2016): 1-11.</a></i></p>

In other words, we will have you look at an example to generate tractogram. Here, a tractogram will be generated using a deterministic algorithm EuDX.  
  
Diffusion tensor imaging (DTI) is one of the most popular MRI techniques to describe the orientation of white matter fibers in brain research. The process of fiber tracking is called tractography. It allows for a virtual dissection and three-dimensional representation of white matter tracts. 
While we could still use FSL for the task, we will have you use <a href="https://dipy.org/">DIPY</a>, a Python package for computational neuroanatomy mainly focusing on diffusion MRI analysis.  
<br>
To generate a tractogram, we need to track the fibers, which is called fiber tracking.
<br>
Local fiber tracking is used to model white matter fibers by creating streamlines from local directional information. In order to perform local fiber tracking, you will apply the following three steps:
<p >
    <ol>
        <li style="font-size: 15px;">Extract directions from diffusion data</li> 
        <li style="font-size: 15px;">Identify when the tracking must stop</li>  
        <li style="font-size: 15px;">Select a set of locations from which to begin tracking</li>
    </ol>
</p>
Combining them will help you obtain a tractography reconstruction!

Ready? Let's go!

## 2.1. Load the data 

In [ ]:
from dipy.core.gradients import gradient_table
from dipy.data import get_fnames
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti, load_nifti_data

hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')
hardi_root_path = op.split(hardi_fname)[0]
label_fname = get_fnames('stanford_labels')

data, affine, hardi_img = load_nifti(hardi_fname, return_img=True)
labels = load_nifti_data(label_fname)
bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)
gtab = gradient_table(bvals, bvecs)

In [ ]:
print('data.shape: ',data.shape)
print('affine.shape: ',affine.shape)
print('hardi_img.shape: ',hardi_img.shape)

print('labels.shape: ',labels.shape)
print('bvals.shape: ',bvals.shape)
print('bvecs.shape: ',bvecs.shape)

In [ ]:
import os.path as op
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

print_dir_tree(hardi_root_path, max_depth=5)

## 2.2. Get the directions from the diffusion data set

### 2.2.1. Defining the white matter region.
Before all else, you will need to visualize the labels above. Run the following cell to load the labels on FSLeyes:

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(label_fname)
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[0]).cmap = 'brain_colours_spectrum'

Based on the values you read within, can you please fill in the cell below with the label corresponding to white matter?

In [ ]:
white_matter_value = 1 # Fill with the value you read in FSLeyes for white matter!

Great, let's now visualize the result of the mask, shall we? For this, let's generate the mask we obtained from above, using our best pal fslmaths ! 
What you should do is extract from the above file the white matter directly, using fslmaths (you can also use the pythonic approach if you want).

In [ ]:
##################
# Solution
# We show the fslmaths approach. Simply set upper and lower threshold to the white matter value in the parcellation.
##################
output_name = op.join(hardi_root_path,'extracted_wm')
subprocess.run(['fslmaths', label_fname, '-thr', str(white_matter_value), '-uthr', str(white_matter_value), '-bin', output_name])
fsleyesDisplay.load(output_name)

That's nice, but we are missing something, aren't we? 
In the coronal slice, look in the middle. There is a region appearing in magenta/purple, but we wanted to include all white matter. Why is that? If you have on top, you'll see it has a different label: 2. This is because this region is white matter, but it is also a sagittal slice of the **corpus callosum**. So we need to do something a bit different. Can you think of a way to modify the above fslmaths command to include both the white matter and the slice of corpus callosum ? :)

In [ ]:
##################
# Solution
# We show the fslmaths approach.
##################

lower_threshold = 1 # Select the lower bound to include both white matter and corpus callosum slice
upper_threshold = 2 # Select the upper bound to include both white matter and corpus callosum slice
output_name = op.join(hardi_root_path,'extracted_wm_complete')

subprocess.run(['fslmaths', label_fname, '-thr', str(lower_threshold), '-uthr', str(upper_threshold), '-bin', output_name])
fsleyesDisplay.load(output_name)

Beautiful! So you can see that these labels can be a bit tricky if you're not careful. Based on your above experience above, we will construct a mask in python directly. Please fill in the cell below the two values for:
- The white matter regions
- The corpus callosum slice

In [ ]:
#######
# Solution
# Remark we create an OR mask between the corpus callosum and white matter values.
# The total mask is simply set from this parcellation.
#######
corpus_callosum_slice_value = 2 # Fill with your value!
white_matter_value = 1 # Fill with your value !

total_white_matter = (labels == corpus_callosum_slice_value) | (labels == white_matter_value)

### 2.2.2. Actually extracting fiber orientations: the orientation distribution function
Okay, now we have a mask to define our fibers. The next cell will be used to estimate the orientation distribution function at each voxel. Before going any further, let's ask why this is necessary. In your opinion, in a single *voxel* how many orientations can we have?

In [ ]:
interactive_MCQ(3,4)

The issue can be summarized as resolving **intravoxel** fiber orientations of MR images.
To summarize these, we use an orientation distribution function, coined ODF.

The ODF is a function that describes how likely local water diffuses in a given orientation in 3D. 

We will not bore you with all mathematical details. What you need to know, however, is that this distribution function will rely on a special model, called the constant solid angle ODF (CSA-ODF) model. The idea is the following: considering the distance from origin of the estimated distribution provides useful information:
<center><img src="imgs/tractography/solid_angle.png"/></center>
<center>Left: odf takes into account solid angle; Right: odf does not take into account solid angle</center>
<i><center>Image taken from <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2911516/">Aganj, Iman, et al. "Reconstruction of the orientation distribution function in single‐and multiple‐shell q‐ball imaging within constant solid angle." Magnetic resonance in medicine 64.2 (2010): 554-566.</a></center></i>

CSA-ODF with solid angle properly accounts for spherical geometry while CSA-ODF without solid angle is less accurate as it doesn’t normalize by solid angle.
 

Let's now estimate the orientation distribution function of each voxel, using the CSA-ODF model. 

In [ ]:
from dipy.reconst.csdeconv import auto_response_ssst
from dipy.reconst.shm import CsaOdfModel
from dipy.data import default_sphere
from dipy.direction import peaks_from_model

# Single fiber response function: the measured signal of a single fiber
# sume: regions where there are single coherent fiber populations
# auto_response_ssst: calculate FA for a ROI of radii equal to roi_radii in the center of the volume
# and return the response function estimated in that region for the voxels with FA higher than 0.7
response, ratio = auto_response_ssst(gtab, data, roi_radii=10, fa_thr=0.7)

# Instantiate the Constant Solid Angle model
csa_model = CsaOdfModel(gtab, sh_order_max=6)

Now that we have our model, the orientation of tract segments can be extracted, looking at the peaks in the model.

In [ ]:
csa_peaks = peaks_from_model(csa_model, data, default_sphere,
                             relative_peak_threshold=.8,
                             min_separation_angle=45,
                             mask=total_white_matter, npeaks=5)

Notice in the above cell the following line:
```python
csa_peaks = peaks_from_model(..., npeaks=5)
```

This means that really, we extract per voxel five peaks at most. This is an important assumption. Depending on your voxel size, you might want to pay attention to this number!

To confirm this, let's have a look at the extracted peak values: 

In [ ]:
csa_peaks.peak_values.shape

Knowing the MR dimensions, you can see that we indeed have five peaks per voxel. Great! Let's visualize it now!

In [ ]:
from fury import actor, window, ui
from ipyvtklink.viewer import ViewInteractiveWidget

In [ ]:
scene = window.Scene()
slice_actor = actor.peak_slicer(csa_peaks.peak_dirs,
                            peaks_values=csa_peaks.peak_values,
                            affine=affine,mask=total_white_matter,)
scene.add(slice_actor)

showm = window.ShowManager(scene, size=(900,900), reset_camera=False)
showm.initialize()
ViewInteractiveWidget(scene.GetRenderWindow())

So as you can see, the orientations do map out to the expected directions!

## 2.3. Set the stop criteria

Now, we need to setup our fiber tracking to stop it. What criterion should we use?
Well, we'll roughly use the idea that when we don't have enough evidence to know where a fiber could have gone, we stop tracking it.
In other words, if there are areas where the diffusion is totally unrestricted (goes in all directions), we have no clue as to where the fiber might continue. For this, we can threshold the tendency of our peaks to depend on a specific direction (anisotropy).<br>
More specifically, we will threshold the general fractional anisotropy (scalar measure between 0 and 1 with 0 → isotropic [e.g CSF, gray matter] and 1 → highly anisotropic [e.g white matter fiber bundles]) of our data to decide when we should stop.

In [ ]:
from dipy.tracking.stopping_criterion import ThresholdStoppingCriterion

stopping_criterion = ThresholdStoppingCriterion(csa_peaks.gfa, .25)

Let's visualize a slice! 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

sli = csa_peaks.gfa.shape[2] // 2
plt.figure('GFA')
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(csa_peaks.gfa[:, :, sli].T, cmap='gray', origin='lower')

plt.subplot(1, 2, 2).set_axis_off()
plt.imshow((csa_peaks.gfa[:, :, sli] > 0.25).T, cmap='gray', origin='lower')

plt.savefig('gfa_tracking_mask.png')

## 2.4. Specify where to begin the fibers tracking

There are different ways to place seeds, ie starting points from which the fiber tracking is started. This depends on the pathways you might like to model! For example, if you only wanted to model the corpus callosum it would not be so interesting to place seeds in other regions of the brain. <br>
So that you understand what the output of the cell below will be, we must first explain what you'll extract in the cell below.<br>
The orientation of an image is described by its affine transformation, if you remember well. Let's call this affine $A$.
A seed point at the center of voxel $[i,j,k]$ will be represented as $[x,y,z, 1]= A \cdot [i,j,k,1]$<br>
In other words, you will get **coordinates in voxel space**. Note that there is one important assumption: the voxels here should be isotropic (the size of a voxel should be same along all directions).

In our specific case, we will start from a sagittal slice of the **corpus callosum**, the one with label 2 to be specific.
Please, create the mask (based on the labels above) to extract **only the slice of corpus callosum with label 2 as a mask**. You can refer to what we did above to do so. 

In [ ]:
from dipy.tracking import utils

seed_mask = labels == 2 # Your code here to extract only the place of interest! 
seeds = utils.seeds_from_mask(seed_mask, affine, density=[2, 2, 2])

## 2.5. Bringing it all together and generating the streamlines
Let's see our ingredients:
- Seeds, generated above and starting from a slice of the corpus callosum
- A mask of regions where we should stop our fibers, based on anisotropy
- Peaks of ODF, at most five peaks per voxel

It remains now to combine all of these to bake so called streamlines (ie: fibers!). To do so, we will use the EuDX algorithm.

In [ ]:
from dipy.tracking.local_tracking import LocalTracking
from dipy.tracking.streamline import Streamlines

# Initialization of LocalTracking. The computation happens in the next step.
streamlines_generator = LocalTracking(csa_peaks, stopping_criterion, seeds,
                                      affine=affine, step_size=.5)
# Generate streamlines object
streamlines_t = Streamlines(streamlines_generator)

In [ ]:
streamlines_t

Beautiful! Let's now visualize our streamlines!
Remember that they represent **only lines that start from the corpus callosum**! 

In [ ]:
from dipy.viz import colormap

# Prepare the display objects.
color = colormap.line_colors(streamlines_t)

streamlines_actor = actor.line(streamlines_t,
                               colors=colormap.line_colors(streamlines_t))

# Create the 3D display.
scene = window.Scene()
scene.add(streamlines_actor)

showm = window.ShowManager(scene, size=(900,900), reset_camera=False)
showm.initialize()
ViewInteractiveWidget(scene.GetRenderWindow())

# Save still images for this static example. Or for interactivity use
#window.record(scene, out_path='tractogram_EuDX.png', size=(800, 800))
#if interactive:
#    window.show(scene)

## 2.6. Store the streamlines into a trackvis file

What if we wanted to save the result as a file? Well, you can! For this, we need to save it to a special format, the TrackVis (.trk) format.

Remember: our goal was to generate the streamlines. It is these streamlines that we therefore want to save! :) 
Let's do it! 

In [ ]:
from dipy.io.stateful_tractogram import Space, StatefulTractogram
from dipy.io.streamline import save_tractogram, save_trk

# This is for the cc slice tractogram
sft = StatefulTractogram(streamlines_t, hardi_img, Space.RASMM)
save_trk(sft, "tractogram_EuDX.trk", streamlines_t)


If you want to visualize it all, you can activate <a href="https://trackvis.org/">TrackVis</a> and open the file from within or re-use `load_trk` from the dipy library.

## 2.7 Conclusions

Let's sum up what we've seen. For a successfull tractography generation, we need the following: 
<table>
    <tr>
        <th>Ingredient</th>
        <th>Role</th>
        <th>How is it created?</th>
    </tr>
    <tr>
        <td>Seeds</td>
        <td>Define starting point of tract propagation.</td>
        <td>Can be done randomly or according to some mask of interest</td>
    </tr>
    <tr>
        <td>Diffusion directions</td>
        <td>Define the local diffusion in a voxel, for all voxels of interest</td>
        <td>Can be done with CSA-ODF or other methods such as e.g structure tensor</td>
    </tr>
    <tr>
        <td>Stopping criteria</td>
        <td>Defines where the tract continues or stops.</td>
        <td>Can be done based on anatomy, information of diffusion direction, combination of both...</td>
    </tr>
    <tr>
        <td>A tracking algorithm</td>
        <td>Combines all ingredients above to generate streamlines</td>
        <td>Line propagation techniques to grow from seed region, or probabilistic with a pdf of fiber orientations.</td>
    </tr>
</table> 


Each of the ingredients can be changed for a different flavour. You can explore <a href="https://docs.dipy.org/stable/examples_built/index">DIPY's tutorials</a> to get an idea of the changes you can operate. Feel free to play around!

## 2.8 Remark: Connectivity analysis based on tractography 

By using the generated streamlines, we could analyze the brain connectivity, for example, which streamlines pass through or not pass through some regions of the brain, how many streamlines are connecting two ROI, etc. To do this, it would be better if we could create a tractography with seeds spaning the entire white matter. Due to RAM concern, we will not do it here. But please feel free to explore it if you are interested! You will find some useful tutorials here:
<a href="https://docs.dipy.org/stable/examples_built/index#streamlines-analysis-and-connectivity">https://docs.dipy.org/stable/examples_built/index#streamlines-analysis-and-connectivity</a>

This ends this short tractography tutorial! Again, do not hesitate to explore more on DIPY's website if you're interested :)
We strongly encourage you to use TrackVis for visualization of tractograms as it's really made for it and is much more intuitive to use for this purpose than FSLeyes.

   


<div class="alert alert-success">
<p><b>🎉 You've reached the end of this week's notebook! Congratulations! 🎉 </b></p>
</div>